In [ ]:
# 각 셀(각 모델)을 돌려 inference 하기 전에 무조건 돌여야 하는 셀

import torch
import cv2
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(0)

# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

def rle_decode_prob(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    values = np.asarray(s[2:][::2], dtype=float)
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.float32)
    value_idx = 0
    for lo, hi in zip(starts, ends):
        img[lo:hi] = values[value_idx]
        value_idx += 1
    return img.reshape(shape)


# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 기존 dataloader - testdata 로딩에 사용
class SatelliteDataset1(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

transform1 = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)



In [ ]:
# upernet-convnext-xlarge

test_dataset = SatelliteDataset1(csv_file='test.csv', transform=transform1, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

# # (저장된 모델이 있다면) 모델 weights 불러오기


from transformers import AutoImageProcessor, UperNetForSemanticSegmentation


model = UperNetForSemanticSegmentation.from_pretrained(
    "openmmlab/upernet-convnext-xlarge", num_labels=1, ignore_mismatched_sizes=True
).to(device)

model.load_state_dict(torch.load("weights/upernet_convNext_xlarge.pth"))



masksarr = []
with torch.no_grad():
    model.eval()
    result = []
    num_images = 0
    for images in tqdm(test_dataloader):
        images = images.float().to(device)

        outputs = model(images).logits
        masks = torch.sigmoid(outputs).cpu().numpy()
        masksarr.append(masks)
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.55).astype(np.uint8) # Threshold

        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

            num_images += 1

submit = pd.read_csv('data/sample_submission.csv')
submit['mask_rle'] = result
submit.to_csv('uper_convnext_xlarge_mask_0.55.csv', index=False)

model = model.to("cpu")

In [ ]:
# unet-convnext-xlarge

test_dataset = SatelliteDataset1(csv_file='test.csv', transform=transform1, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

# # (저장된 모델이 있다면) 모델 weights 불러오기

from backbones_unet.model.unet import Unet
model = Unet(
    backbone='convnext_xlarge_in22ft1k',
    in_channels=3,
    num_classes=1,
).to(device=device)


model.load_state_dict(torch.load("weights/unet_convNext_xlarge_final.pth"))

masksarr = []
with torch.no_grad():
    model.eval()
    result = []
    num_images = 0
    for images in tqdm(test_dataloader):
        images = images.float().to(device)

        outputs = model(images)
        masks = torch.sigmoid(outputs).cpu().numpy()
        masksarr.append(masks)
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.55).astype(np.uint8) # Threshold

        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

            num_images += 1

submit = pd.read_csv('data/sample_submission.csv')
submit['mask_rle'] = result
submit.to_csv('unet_convnext_xlarge_mask_0.55.csv', index=False)
model = model.to("cpu")

In [ ]:
# unet-convnext-large

test_dataset = SatelliteDataset1(csv_file='test.csv', transform=transform1, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

# # (저장된 모델이 있다면) 모델 weights 불러오기

from backbones_unet.model.unet import Unet
model = Unet(
    backbone='convnext_large_in22ft1k',
    in_channels=3,
    num_classes=1,
).to(device=device)


model.load_state_dict(torch.load("weights/unet_convNext_large_final.pth"))

masksarr = []
with torch.no_grad():
    model.eval()
    result = []
    num_images = 0
    for images in tqdm(test_dataloader):
        images = images.float().to(device)

        outputs = model(images)
        masks = torch.sigmoid(outputs).cpu().numpy()
        masksarr.append(masks)
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.6).astype(np.uint8) # Threshold

        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

            num_images += 1
submit = pd.read_csv('data/sample_submission.csv')
submit['mask_rle'] = result
submit.to_csv('unet_convnext_large_mask_0.6.csv', index=False)
model = model.to("cpu")

In [ ]:
# unet_convNext_base

test_dataset = SatelliteDataset1(csv_file='test.csv', transform=transform1, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

# # (저장된 모델이 있다면) 모델 weights 불러오기

from backbones_unet.model.unet import Unet
model = Unet(
    backbone='convnext_base_in22ft1k',
    in_channels=3,
    num_classes=1,
).to(device=device)


model.load_state_dict(torch.load("weights/unet_convNext_base_final.pth"))

masksarr = []
with torch.no_grad():
    model.eval()
    result = []
    num_images = 0
    for images in tqdm(test_dataloader):
        images = images.float().to(device)

        outputs = model(images)
        masks = torch.sigmoid(outputs).cpu().numpy()
        masksarr.append(masks)
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.6).astype(np.uint8) # Threshold

        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

            num_images += 1

submit = pd.read_csv('data/sample_submission.csv')
submit['mask_rle'] = result
submit.to_csv('unet_convnext_base_mask_0.6.csv', index=False)
model = model.to("cpu")

In [ ]:
# uper_convnext_large

test_dataset = SatelliteDataset1(csv_file='test.csv', transform=transform1, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

# # (저장된 모델이 있다면) 모델 weights 불러오기

from backbones_unet.model.unet import Unet

from transformers import AutoImageProcessor, UperNetForSemanticSegmentation
model = UperNetForSemanticSegmentation.from_pretrained(f"openmmlab/upernet-convnext-large", num_labels=1, ignore_mismatched_sizes=True).to(device)


model.load_state_dict(torch.load("weights/upernet_convNext_large_final.pth"))

masksarr = []
with torch.no_grad():
    model.eval()
    result = []
    num_images = 0
    for images in tqdm(test_dataloader):
        images = images.float().to(device)

        outputs = model(images).logits
        masks = torch.sigmoid(outputs).cpu().numpy()
        masksarr.append(masks)
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.6).astype(np.uint8) # Threshold

        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

            num_images += 1

submit = pd.read_csv('data/sample_submission.csv')
submit['mask_rle'] = result
submit.to_csv('uper_convnext_large_mask_0.6.csv', index=False)
model = model.to("cpu")

In [ ]:
# 에피션트넷

test_dataset = SatelliteDataset1(csv_file='test.csv', transform=transform1, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

# # (저장된 모델이 있다면) 모델 weights 불러오기

from backbones_unet.model.unet import Unet
model = Unet(
    backbone='tf_efficientnetv2_l_in21ft1k',
    in_channels=3,
    num_classes=1,
).to(device=device)

model.load_state_dict(torch.load("weights/unet_efficientnetV2_l_final.pth"))

masksarr = []
with torch.no_grad():
    model.eval()
    result = []
    num_images = 0
    for images in tqdm(test_dataloader):
        images = images.float().to(device)

        outputs = model(images)
        masks = torch.sigmoid(outputs).cpu().numpy()
        masksarr.append(masks)
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.6).astype(np.uint8) # Threshold

        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

            num_images += 1

submit = pd.read_csv('data/sample_submission.csv')
submit['mask_rle'] = result
submit.to_csv('unet_efficientnetV2_l_mask_0.6.csv', index=False)
model = model.to("cpu")